## Semi-Structured RAG For Private Data

In this homework assignment, you will be delving into the realm of **Retrieval Augmented Generation (RAG)**.

Your objective is to construct a system that leverages retrieval from a **private database** consisting of PDFs. These PDFs encapsulate a rich variety of content, including textual information, images, and tables.

The challenge lies in preserving all these components while efficiently extracting relevant data based on a user's input question.

- As a first step, you will need to develop mechanisms for extracting text from the PDFs. Also, extract textual embeddings for following comparison with the user's input.
- Subsequently, you should implement a process to identify and retrieve the most pertinent information matching a user's query.
- Because some input texts are too long, we have to summarize them, and then use the summary of the most similar text to LLM as input.
- Then, you will integrate this retrieved information with a Large Language Model (LLM) to generate comprehensive and contextually relevant responses to user queries.
- Finally, you will apply this mechanism in a Multimodal approach, where you convert PDF images to clip embeddings and use the input's textual CLIP embeddings to compare with the ground truth's image embeddings and find the most similar image to the input text.
- As we are using Unimodal LLMs, we can not give those images to the LLM. Hence, we use image captions to be used in LLM's input.

This holistic approach ensures that no valuable information is lost, and the system provides nuanced answers by combining both the knowledge embedded in the PDFs and the capabilities of the LLM.

Instruction:

<font color='77CC99'>Follow the Green texts and fill out the notebook.</font>


<img src='https://drive.google.com/uc?id=1kODk16WWrn9DqvaWoEAekHRXup1djGjl' width="75%">

## Packages

In [1]:
# restart kernel after first instllation
%%capture
!apt-get install -y poppler-utils
!apt-get install tesseract-ocr
!pip install pytesseract
# for image extraction from pdf
! pip install PyMuPDF
! pip install Pillow
# text embedding
! pip install -U sentence-transformers
! pip install transformers accelerate bitsandbytes>=0.39.0 -q

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler118 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler118 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 78 not upgraded.
Need to get 3428 kB of archives.
After this operation, 17.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 poppler-data all 0.4.11-1 [2171 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.3 [1072 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 3428 kB in 2s (2192 kB/s)       
Selecting previously unselected

# 0 - Loading Data

### 0.1 - Downoading the PDF

In [2]:
from pathlib import Path
import urllib.request

# Define the name of the PDF file and then download them
file_name = "Dall_E_paper"

url = "https://arxiv.org/pdf/2204.06125.pdf"
file_path = f"{file_name}.pdf"
urllib.request.urlretrieve(url, file_path)

('Dall_E_paper.pdf', <http.client.HTTPMessage at 0x7daeb7c60550>)

## 0.2 - Extract Images and Texts

Implement mechanisms to extract images and texts from the downloaded PDFs.

In [3]:
!which pdftotext

/usr/bin/pdftotext


In [4]:
import pytesseract
print(pytesseract.get_tesseract_version())

4.1.1


In [5]:
# Import required dependencies
import fitz
import os
from PIL import Image

#### Step 0.2.1: Extract and Store Images

In [6]:
# Open PDF file
pdf_file = fitz.open(file_path)

# Calculate number of pages in PDF file
page_nums = len(pdf_file)

# Create empty list to store images information
images_list = []

# Extract all images information from each page
for page_num in range(page_nums):
    page_content = pdf_file[page_num]
    images_list.extend(page_content.get_images())

In [7]:
images_path = "./images/"
Path(images_path).mkdir(parents=True, exist_ok=True)

#Save all the extracted images
for i, image in enumerate(images_list, start=1):
    #Extract the image object number
    xref = image[0]
    #Extract image
    base_image = pdf_file.extract_image(xref)
    #Store image bytes
    image_bytes = base_image['image']
    #Store image extension
    image_ext = base_image['ext']
    #Generate image file name
    image_name = file_name + '_' +str(i) + '.' + image_ext
    #Save image
    with open(os.path.join(images_path, image_name) , 'wb') as image_file:
        image_file.write(image_bytes)
        image_file.close()

### Step 0.2.2: Extract and Store Texts From PDF Content

In [8]:
!pip install unstructured[all-docs]==0.11.2

  Obtaining dependency information for unstructured[all-docs]==0.11.2 from https://files.pythonhosted.org/packages/f8/3d/561a78b6160337f3a8607c183a6ae72352b0b5278b3ff309093c1b3cd211/unstructured-0.11.2-py3-none-any.whl.metadata
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for python-iso639 from https://files.pythonhosted.org/packages/d4/3e/f20c6edf8b95db750a52269d3dffd6e6a4432989cc7731efef2a06c1d187/python_iso639-2023.12.11-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for python-docx>=1.1.0 from https://files.pythonhosted.org/packages/5f/d8/6948f7ac00edf74bfa52b3c5e3073df20284bec1db466d13e668fe991707/python_docx-1.1.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━

In [9]:
from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf

path='./'

# Specify the path to the poppler installation
poppler_path = '/usr/bin/'  # Replace with the path obtained from the previous step

# Specify the path to the Tesseract OCR installation
tesseract_path = '/usr/bin/tesseract'  # Replace with the path obtained from the previous step


# Get elements
raw_pdf_elements = partition_pdf(
    filename= "./"+"Dall_E_paper.pdf",
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 1900 chars
    # Attempt to keep chunks > 1000 chars
    # Hard max on chunks
    max_characters=2000,
    new_after_n_chars=1900,
    combine_text_under_n_chars=1000,
    image_output_dir_path=poppler_path,
    tesseract_path=tesseract_path,
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Text
text_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.CompositeElement" in str(type(element)):
      text_elements.append(str(element))

print(len(text_elements))

39


Because some texts are too long, we have to summarize them at first

In [11]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summarized_text_elements = summarizer(text_elements , max_length=100, do_sample=False)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Your max_length is set to 100, but your input_length is only 37. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 100, but your input_length is only 50. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


# 1 - Unimodal RAG

## 1.1 - Loading True Text Data as Embeded Vectors

In this section, we should convert the text data into embedding vectors and store them. Hence, in the following step. having an input, by comparing we can find out the most similar fact with the input.

We use this model for [Text-Embedding](https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2)

In [12]:
from sentence_transformers import SentenceTransformer, util

text_emb_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [13]:
text_embeddings = text_emb_model.encode(text_elements, convert_to_tensor=True)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Now, we have all our crucial embeddings. Thus, if we have a new input, we know that we should compare the input's embeddings with the text_embeddings element and find the closest one.

## 1.2 - Unimodal Semi-Structured RAG

### Step 1.2.1: Most Similar Ground Truth Text Ectraction

At the fist step, for any given input, we have to have evaluation functions to find the closest embedding vector to the input vectors. We use the Cosine similarity for this operation.

<font color='77CC99'>Write a function "text_embedding_similarity" to convert input texts to embedded vector and then returns the similarity between the input text and any of the ground truth texts.</font>


In [14]:
import numpy as np

def get_similarity(embeddings_1, embeddings_2):

  embeddings_1 = embeddings_1 / embeddings_1.norm(dim=-1, keepdim=True)
  embeddings_2 = embeddings_2 / embeddings_2.norm(dim=-1, keepdim=True)

  return np.array(embeddings_1.cpu().detach().numpy() @ embeddings_2.cpu().detach().numpy().T, dtype='float32')

In [15]:
def text_embedding_similarity(input_text, text_embeddings, text_emb_model):

  ### To Do ###

  input_text_emb = text_emb_model.encode(input_text, convert_to_tensor=True)

  ### End ###

  return get_similarity(text_embeddings, input_text_emb)

In [16]:
input_text = "is DALL-E2 uses a clip model inside?"
text_embedding_similarity(input_text, text_embeddings, text_emb_model)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([0.18537408, 0.2028819 , 0.21293405, 0.20692347, 0.2686563 ,
       0.31146654, 0.01873748, 0.21311939, 0.20436674, 0.25198755,
       0.26972088, 0.24603578, 0.25822914, 0.25395477, 0.23482764,
       0.23657548, 0.17059219, 0.15917063, 0.15459308, 0.14453976,
       0.1123264 , 0.18208526, 0.12710601, 0.37942076, 0.05238137,
       0.32438618, 0.3001934 , 0.06964831, 0.10325839, 0.1321447 ,
       0.02927516, 0.15812719, 0.10174508, 0.2626279 , 0.15325926,
       0.00922203, 0.3108288 , 0.04998646, 0.12929474], dtype=float32)

<font color='77CC99'> Now, write a function that finds "Summaries" of the k most similar ground truth texts to the user's input. function "text_retrival"</font>

In [17]:
import heapq

def text_retrival(k, input_text, text_embeddings, text_elements, summarized_text_elements, text_emb_model):

  ### To Do ###
  similarities = text_embedding_similarity(input_text, text_embeddings, text_emb_model)
  k_similar_indices = heapq.nlargest(k, range(len(similarities)), similarities.take)
  selected_text_elements = [{"text": text_elements[i], "summary": summarized_text_elements[i]} for i in k_similar_indices]
  ### End ###

  return {"selected_text_elements":selected_text_elements}

In [18]:
# Example usage
k = 3  # You can set the value of k as per your requirement
input_text = "is DALL-E2 uses a clip model inside?"

result = text_retrival(k, input_text, text_embeddings, text_elements, summarized_text_elements, text_emb_model)
print(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'selected_text_elements': [{'text': 'Since its release, CLIP [39] has been used extensively to steer generative image models towards text prompts. Galatolo et al., Patashnik et al., Murdock, Gal et al. [19, 36, 32, 18] guide GANs using gradients from a CLIP model. For diffusion models, Dhariwal and Nichol [11] introduced classiﬁer guidance as a way to use gradients from a classiﬁer trained on noised images to steer the model towards higher quality generations. Nichol et al. [35] train a CLIP model on noised images and guide a text-conditional diffusion model, while Crowson, Crowson [7, 8] use an unnoised CLIP model to guide unconditional or class-conditional diffusion models. Ho and Salimans [24] introduced classiﬁer-free guidance and showed that one can perform guidance\n\n15\n\nal v % ¥\n\n(a) unCLIP  (b) GLIDE \n\nFigure 14: Samples from unCLIP and GLIDE for the prompt “a red cube on top of a blue cube”.\n\nimplictly from the predictions of the model with and without the conditioni

### Step 1.2.2: Load the core LLM and Combine them all

We use a Question-answering model as the core of our system. In fact, having the input text and finding the closest ground truth fact to the input text, we can give them both to an LLM to answer the question.

Here we load the core LLM for our Unimodal  Semi-Structured RAG. [Model in HF](https://huggingface.co/samwit/koala-7b)

In [19]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
import textwrap

model = LlamaForCausalLM.from_pretrained(
    "samwit/koala-7b",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00014.bin:   0%|          | 0.00/1.96G [00:00<?, ?B/s]

pytorch_model-00002-of-00014.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00003-of-00014.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [36]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15)

tokenizer.pad_token_id = tokenizer.eos_token_id

Now, what follows is our Prompt, based on that, do the task bellow.


<font color='77CC99'> Based on the prompt and what we have done before, write a function that answers the user's question by finding the most related ground truth text(fact) by giving the prompt to LLM. Function "Unimodal_Question_Answering" </font>

In [22]:
# Prompt
prompt_text = """ANSWER the QUESTION in conformity to on FACTS. \n
FACTS: \n {text_facts} \n
QUESTION: {user_question} \n
ANSWER:  """

In [23]:
def Unimodal_Question_Answering(input_text, k=1):

  ### To Do ###
  facts = ''
  k_relevant_facts = text_retrival(k, input_text, text_embeddings, text_elements, summarized_text_elements, text_emb_model)['selected_text_elements']
  for i, relevant_fact in enumerate(k_relevant_facts):
    facts += relevant_fact['summary']['summary_text']
    if i != k-1:
      facts += '\n'

  prompt = prompt_text.format(text_facts=facts, user_question=input_text)
  response = pipeline(prompt)
  answer_idx = response[0]['generated_text'].split().index('ANSWER:')
  response = ' '.join(response[0]['generated_text'].split()[answer_idx+1:])
  ### End ###

  return response

In [24]:
input_text = "is DALL-E2 uses a clip model inside?"

response = Unimodal_Question_Answering(input_text, k=1)
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No, DALL-E2 does not use a CLIP model inside.


# 2 - Multimodal RAG

In this section, we want to add another modality to our unimodal RAG. What happens if we can consider images as ground truth facts?

We have stored all ground truth images. Thus, in this step, we should extract image embeddings for comparison with textual input embeddings

## 2.1 - Loading CLIP Model for Extracting Embeddings

<font color='77CC99'> Load CLIP model for extracting textual and visial embeddings, then convert all input images to their corresponding vectors.

[Huggingface Link](https://huggingface.co/docs/transformers/model_doc/clip) </font>


In [25]:
from PIL import Image
import requests
from transformers import AutoTokenizer, CLIPTextModelWithProjection
from transformers import AutoProcessor, CLIPVisionModelWithProjection

### To Do ###

textual_clip_model = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
textual_clip_tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
visual_clip_model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
visual_clip_processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

### End ###

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

In [26]:
from tqdm import tqdm
import torch
import glob
images_path = glob.glob('./images/*')

### To Do ###
images_embeddings = []
for image_path in tqdm(images_path):
  image = Image.open(image_path)
  inputs = visual_clip_processor(images=image, return_tensors="pt")
  outputs = visual_clip_model(**inputs)
  image_embeds = outputs.image_embeds
  images_embeddings.append(image_embeds.squeeze())

images_embeddings = torch.stack(images_embeddings)

### End ###

100%|██████████| 103/103 [00:18<00:00,  5.62it/s]


As we are using unimodsl LLM, we need to make image's information understandable for LLM. Hence, we extract textual information of imaged as "Caption" store them in "captions" list.

<font color='77CC99'>Write the corresponding code.</font>


[Image Captioning HF Model](https://huggingface.co/nlpconnect/vit-gpt2-image-captioning)

In [27]:
from transformers import pipeline
image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

### To Do ###

caption_list = []
for image_path in tqdm(images_path):
  caption_list.append(image_to_text(image_path)[0]['generated_text'])

### End ###

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
  0%|          | 0/103 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_i

Ok, now that we have every thing ready, we can code the Multimodal Semi-Structured RAG

## 2.2 - Multimodal Semi-Structured RAG

### Step 2.2.1: Most Similar Ground Truth Image Ectraction

At the fist step, for any given input, we have to have evaluation functions to find the closest visual embedding vector to the input's textual vectors. We use the Cosine similarity for this operation.

<font color='77CC99'>Write a function "visual_embedding_similarity" to convert input texts to clip embedding vector and then returns the similarity between the input text and any of the ground truth images.</font>

Note: You can use "get_similarity" function that you have definced before.

In [28]:
def get_embedding_similarity(input_text, images_embeddings, textual_clip_tokenizer, textual_clip_model):

  ### To Do ###
  inputs = textual_clip_tokenizer(input_text, padding=True, return_tensors="pt")
  outputs = textual_clip_model(**inputs)
  text_embeds = outputs.text_embeds
  ### End ###

  return get_similarity(images_embeddings, text_embeds)

<font color='77CC99'>Now, write a function that finds k most similar Text/Image to user's input.</font>

In [29]:
import heapq

def multimodal_retrival(k, input_text, text_embeddings, text_elements, summarized_text_elements,
                        text_emb_model, images_embeddings, caption_list, textual_clip_tokenizer , textual_clip_model):

  ### To Do ###
  text_similarities = text_embedding_similarity(input_text, text_embeddings, text_emb_model)
  k_similar_text_indices = heapq.nlargest(k, range(len(text_similarities)), text_similarities.take)

  image_similarities = get_embedding_similarity(input_text, images_embeddings, textual_clip_tokenizer, textual_clip_model)
  k_similar_image_indices = heapq.nlargest(k, range(len(image_similarities)), image_similarities.take)

  selected_text_elements = [{"text": text_elements[i], "summary": summarized_text_elements[i]} for i in k_similar_text_indices]
  selected_image_elements = [{"caption": caption_list[i]} for i in k_similar_image_indices]

  return {"selected_image_elements": selected_image_elements,
          "selected_text_elements": selected_text_elements}

  ### End ###

### Step 2.2.2: Use the core LLM and Combine them all

In this section, based on what we have done before(Loading LLM), we want to use what we have done in this section to write the Multimodal RAG. Do it as follows.

<font color='77CC99'> Based on the new prompt which contains both textual ground truth facts and the caption of visual ground truth images, to write the "Multimodal_Question_Answering" function. This function should takes the user's textual question as input, then finds the most correlated textual and visual ground truth. Then gives them all to LLM via prompt.</font>

In [30]:
# Prompt
prompt_text = """ANSWER the QUESTION in conformity to on FACTS. \n
FACTS: \n {text_facts} \n {image_facts}. \n
QUESTION: {user_question} \n
ANSWER:  """

In [39]:
def Multimodal_Question_Answering(input_text,k=1):

  ### To Do ###
  facts, i_facts = '', ''
  k_relevant_facts = multimodal_retrival(k, input_text, text_embeddings, text_elements, summarized_text_elements,
                        text_emb_model, images_embeddings, caption_list, textual_clip_tokenizer , textual_clip_model)
  k_relevant_text_facts = k_relevant_facts['selected_text_elements']
  k_relevant_image_facts = k_relevant_facts['selected_image_elements']
  for i, (relevant_text_fact, relevant_image_fact)  in enumerate(zip(k_relevant_text_facts, k_relevant_image_facts)):
    facts += relevant_text_fact['summary']['summary_text']
    i_facts += relevant_image_fact['caption']
    if i != k-1:
      facts += '\n'
      i_facts += '\n'

  prompt = prompt_text.format(text_facts=facts, image_facts=i_facts, user_question=input_text)
  response = pipeline(prompt)
  answer_idx = response[0]['generated_text'].split().index('ANSWER:')
  response = ' '.join(response[0]['generated_text'].split()[answer_idx+1:])
  ### End ###

  return response

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15)

tokenizer.pad_token_id = tokenizer.eos_token_id

In [40]:
input_text = "is DALL-E2 uses a clip model inside?"

response = Multimodal_Question_Answering(input_text,k=1)
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No, DALL-E2 does not use a CLIP model inside.


<font color='77CC99'>The Answer to the input question is "Yes" or "No". What are your Semi-structured models' answers? (Both Unimodal and Multimodal). Are they right or not?</font>

<font color='CC7799'>They both answered no, which is wrong.</br> Besides, in multimodal case, the generated captions using images
provided in the paper seems irrelevant and I think they can't give the model any additional hint.</font>